# EDA de **Telefonía móvil: Ingresos**

Cuaderno preparado para ejecutarse en **Visual Studio/VS Code** con Python. Incluye pasos guiados y visualizaciones básicas.
**Fuente de datos:** CSV con columnas `Registro_ID, Año, Trimestre, Ingresos, TiempoTrim_ID`.

## 1. Importaciones y configuración
- Se importan `pandas`, `numpy`, `matplotlib` y `seaborn`.
- Se configura el estilo de gráficos y se asegura codificación UTF-8.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (10, 5),
    'axes.titlesize': 14,
    'axes.labelsize': 12
})

## 2. Carga del dataset desde la ruta indicada
> **Ruta proporcionada:** `C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Ingresos_con_ID.csv`

Si tu usuario o carpeta cambian, actualiza la cadena `ruta`. Se preserva la codificación **UTF‑8 con BOM** para abrir acentos correctamente en Excel.

In [ ]:
ruta = r"C:\\Users\\u607935\\OneDrive - Telecom Argentina SA\\Escritorio\\Curso de Data Analitycs\\Proyencto Integrador Final Grupo 7\\Comunicaciones  Moviles\\Telefonía móvil Ingresos_con_ID.csv"

# Cargar el archivo desde la ruta de Windows indicada
df = pd.read_csv(ruta, encoding='utf-8-sig')

# Copia de seguridad del crudo
df_raw = df.copy()

# Vista rápida
display(df.head())
print('
Filas, Columnas:', df.shape)
print('
Columnas:', list(df.columns))

## 3. Perfil inicial del data set
- Tipos de datos y memoria (`df.info()`).
- Valores faltantes y duplicados.
- Estadísticos descriptivos de `Ingresos`.

In [ ]:
# Tipos de datos
df.info()

# Valores faltantes y duplicados
print('
Valores faltantes por columna:
', df.isna().sum())
print('
Duplicados (filas completas):', df.duplicated().sum())

# Estadísticos descriptivos de Ingresos
if 'Ingresos' in df.columns:
    print('
Estadísticos Ingresos:')
    print(df['Ingresos'].describe())

## 4. Validaciones de consistencia
- Conversión a numérico.
- Chequeo de negativos y valores extremos (IQR).
- Construcción de `Periodo` (YYYY-Qn).

In [ ]:
numeric_cols = ['Registro_ID','Año','Trimestre','Ingresos','TiempoTrim_ID']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Negativos
print('
Negativos por columna:
', (df[numeric_cols] < 0).sum())

# Outliers por IQR en Ingresos
if 'Ingresos' in df.columns:
    q1 = df['Ingresos'].quantile(0.25)
    q3 = df['Ingresos'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    outliers = df[(df['Ingresos'] < lower) | (df['Ingresos'] > upper)]
    print(f'Outliers detectados en Ingresos: {len(outliers)}')

# Periodo YYYY-Qn
df['Periodo'] = pd.PeriodIndex(year=df['Año'], quarter=df['Trimestre']).astype(str)

## 5. Limpieza y ordenamiento
- Eliminación de duplicados.
- Orden cronológico por `Año` y `Trimestre`.
- (Opcional) Filtrado de outliers para análisis (sin borrar del dataset principal).

In [ ]:
df = df.drop_duplicates()
df = df.sort_values(['Año','Trimestre']).reset_index(drop=True)

# Crear vista sin outliers (no se reemplaza df principal)
if 'Ingresos' in df.columns:
    q1 = df['Ingresos'].quantile(0.25)
    q3 = df['Ingresos'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    df_no_out = df[(df['Ingresos'] >= lower) & (df['Ingresos'] <= upper)].copy()
    print('Filas sin outliers (vista):', df_no_out.shape)

## 6. Variables derivadas
- **`Ingresos_QoQ_%`**: variación trimestral.
- **`Ingresos_YoY_%`**: variación interanual (t-4).
- **`Ingresos_MovAvg4`**: media móvil de 4 trimestres.

In [ ]:
df['Ingresos_QoQ_%'] = df['Ingresos'].pct_change() * 100
df['Ingresos_YoY_%'] = df['Ingresos'].pct_change(4) * 100
df['Ingresos_MovAvg4'] = df['Ingresos'].rolling(4).mean()

display(df[['Año','Trimestre','Ingresos','Ingresos_QoQ_%','Ingresos_YoY_%','Ingresos_MovAvg4']].tail(8))

## 7. Visualizaciones
- Serie temporal de `Ingresos` y su media móvil (4 trimestres).
- Variación `Ingresos_YoY_%` y `Ingresos_QoQ_%`.

In [ ]:
import matplotlib.ticker as mtick

# 7.1 Ingresos y media móvil
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Ingresos', marker='o', label='Ingresos', color='#2E86C1')
sns.lineplot(data=df, x='Periodo', y='Ingresos_MovAvg4', marker='o', label='Media móvil (4)', color='#7D3C98')
plt.xticks(rotation=45, ha='right')
plt.title('Ingresos trimestrales (con media móvil 4)')
plt.xlabel('Periodo')
plt.ylabel('Ingresos')
plt.legend()
plt.tight_layout()
plt.show()

# 7.2 Variación YoY y QoQ
plt.figure()
sns.lineplot(data=df, x='Periodo', y='Ingresos_YoY_%', marker='o', label='YoY %', color='#1ABC9C')
sns.lineplot(data=df, x='Periodo', y='Ingresos_QoQ_%', marker='o', label='QoQ %', color='#E67E22')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter())
plt.xticks(rotation=45, ha='right')
plt.title('Variación de Ingresos (YoY y QoQ)')
plt.xlabel('Periodo')
plt.ylabel('%')
plt.legend()
plt.tight_layout()
plt.show()

## 8. Guardado del archivo limpio
Se guarda el CSV **en la misma ruta** con la **leyenda `_limpio`** agregada al nombre, conservando la codificación `utf-8-sig`.

In [ ]:
ruta_path = Path(ruta)
ruta_limpia = ruta_path.with_name(ruta_path.stem + '_limpio' + ruta_path.suffix)
df.to_csv(ruta_limpia, index=False, encoding='utf-8-sig')
print('Guardado en:', ruta_limpia)